In [1]:
import pandas as pd
import regex as re
from sklearn.metrics.pairwise import cosine_similarity
import click


In [2]:
url = "https://drive.google.com/file/d/1atSUefnKp-VxShl_fd0slilQ3sbHGEiP/view?usp=share_link"
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
movies_df = pd.read_csv(path)

url = "https://drive.google.com/file/d/1ASPgCecn2k4w90uIDOh3Gtw2H77QpoGv/view?usp=share_link"
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
ratings_df = pd.read_csv(path)

In [3]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [5]:
ratings_movies = ratings_df.groupby('movieId').agg({"userId":"count", "rating":"mean"})
ratings_movies = ratings_movies.reset_index()
ratings_movies.rating = ratings_movies.rating.round(2)
ratings_movies.head()

,movieId,userId,rating
0,1,215,3.92
1,2,110,3.43
2,3,52,3.26
3,4,7,2.36
4,5,49,3.07


In [6]:
ratings_movies = ratings_movies.merge(movies_df, left_on='movieId', right_on='movieId')
ratings_movies.genres = ratings_movies.genres.str.replace('|',',')
ratings_movies.head()

/var/folders/kq/fhf47yp54w1byy_p5j5n4c340000gn/T/ipykernel_34953/2885329868.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  ratings_movies.genres = ratings_movies.genres.str.replace('|',',')


,movieId,userId,rating,title,genres
0,1,215,3.92,Toy Story (1995),"Adventure,Animation,Children,Comedy,Fantasy"
1,2,110,3.43,Jumanji (1995),"Adventure,Children,Fantasy"
2,3,52,3.26,Grumpier Old Men (1995),"Comedy,Romance"
3,4,7,2.36,Waiting to Exhale (1995),"Comedy,Drama,Romance"
4,5,49,3.07,Father of the Bride Part II (1995),Comedy


In [7]:
ratings_movies = ratings_movies[['movieId', 'title', 'userId', 'rating', 'genres']]
ratings_movies = ratings_movies.rename(columns={
    'title': 'Title',
    'userId':'Total Reviews',
    'rating':'Rating',
    'genres':'Genre'
})

ratings_movies.head()

,movieId,Title,Total Reviews,Rating,Genre
0,1,Toy Story (1995),215,3.92,"Adventure,Animation,Children,Comedy,Fantasy"
1,2,Jumanji (1995),110,3.43,"Adventure,Children,Fantasy"
2,3,Grumpier Old Men (1995),52,3.26,"Comedy,Romance"
3,4,Waiting to Exhale (1995),7,2.36,"Comedy,Drama,Romance"
4,5,Father of the Bride Part II (1995),49,3.07,Comedy


# Popular


In [8]:
ratings_movies.nlargest(20, 'Total Reviews').sort_values(by='Rating', ascending=False)

,movieId,Title,Total Reviews,Rating,Genre
277,318,"Shawshank Redemption, The (1994)",317,4.43,"Crime,Drama"
2224,2959,Fight Club (1999),218,4.27,"Action,Crime,Drama,Thriller"
46,50,"Usual Suspects, The (1995)",204,4.24,"Crime,Mystery,Thriller"
224,260,Star Wars: Episode IV - A New Hope (1977),251,4.23,"Action,Adventure,Sci-Fi"
461,527,Schindler's List (1993),220,4.22,"Drama,War"
897,1196,Star Wars: Episode V - The Empire Strikes Back...,211,4.22,"Action,Adventure,Sci-Fi"
899,1198,Raiders of the Lost Ark (Indiana Jones and the...,200,4.21,"Action,Adventure"
257,296,Pulp Fiction (1994),307,4.20,"Comedy,Crime,Drama,Thriller"
1938,2571,"Matrix, The (1999)",278,4.19,"Action,Sci-Fi,Thriller"
314,356,Forrest Gump (1994),329,4.16,"Comedy,Drama,Romance,War"


In [9]:
ratings_movies.loc[ratings_movies['Title'].str.contains("(\d{4})"), 'Released Year'] = ratings_movies.Title.str[-6:]
ratings_movies['Released Year'] = ratings_movies['Released Year'].str.replace('\D', '')
ratings_movies['Released Year'] = ratings_movies['Released Year'].fillna('Unknown')

/var/folders/kq/fhf47yp54w1byy_p5j5n4c340000gn/T/ipykernel_34953/924320907.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  ratings_movies.loc[ratings_movies['Title'].str.contains("(\d{4})"), 'Released Year'] = ratings_movies.Title.str[-6:]
/var/folders/kq/fhf47yp54w1byy_p5j5n4c340000gn/T/ipykernel_34953/924320907.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  ratings_movies['Released Year'] = ratings_movies['Released Year'].str.replace('\D', '')


In [10]:
def popular_movies():
    n = int(input('How many movies do you want to see ? '))
    ratings_movies_top_20_popular = ratings_movies.nlargest(38, 'Total Reviews').sort_values(by='Rating', ascending=False)
    return ratings_movies_top_20_popular[['Title', 'Total Reviews', 'Rating', 'Released Year']].head(n)

In [11]:
popular_movies()

,Title,Total Reviews,Rating,Released Year
277,"Shawshank Redemption, The (1994)",317,4.43,1994
659,"Godfather, The (1972)",192,4.29,1972
2224,Fight Club (1999),218,4.27,1999
46,"Usual Suspects, The (1995)",204,4.24,1995
224,Star Wars: Episode IV - A New Hope (1977),251,4.23,1977
461,Schindler's List (1993),220,4.22,1993
897,Star Wars: Episode V - The Empire Strikes Back...,211,4.22,1980
899,Raiders of the Lost Ark (Indiana Jones and the...,200,4.21,1981
257,Pulp Fiction (1994),307,4.20,1994
1938,"Matrix, The (1999)",278,4.19,1999


# Genres


In [12]:
ratings_movies.head()

,movieId,Title,Total Reviews,Rating,Genre,Released Year
0,1,Toy Story (1995),215,3.92,"Adventure,Animation,Children,Comedy,Fantasy",1995
1,2,Jumanji (1995),110,3.43,"Adventure,Children,Fantasy",1995
2,3,Grumpier Old Men (1995),52,3.26,"Comedy,Romance",1995
3,4,Waiting to Exhale (1995),7,2.36,"Comedy,Drama,Romance",1995
4,5,Father of the Bride Part II (1995),49,3.07,Comedy,1995


In [13]:
def genre_selection():
    input_genre = input('What genre do you want to watch?')
    n = int(input('How many movies do you want us to recommend you?'))

    for i in ratings_movies.Genre:
        if i == input_genre:
            top_genre = ratings_movies.loc[ratings_movies.Genre.str.contains(i),['Title', 'Rating', 'Total Reviews']].nlargest(n, 'Total Reviews').sort_values('Rating', ascending=False)
    return top_genre

In [14]:
genre_selection()

,Title,Rating,Total Reviews
2224,Fight Club (1999),4.27,218
224,Star Wars: Episode IV - A New Hope (1977),4.23,251
897,Star Wars: Episode V - The Empire Strikes Back...,4.22,211
899,Raiders of the Lost Ark (Indiana Jones and the...,4.21,200
1938,"Matrix, The (1999)",4.19,278
910,Star Wars: Episode VI - Return of the Jedi (1983),4.14,196
97,Braveheart (1995),4.03,237
507,Terminator 2: Judgment Day (1991),3.97,224
418,Jurassic Park (1993),3.75,238
615,Independence Day (a.k.a. ID4) (1996),3.45,202


In [46]:
Genre_all = ratings_movies.Genre

In [49]:
Genre_all = list(Genre_all.str.split(','))

In [51]:
list_genre= []
for i in Genre_all:
    for j in i:
        list_genre.append(j)

In [61]:
genre_df = pd.DataFrame(list_genre).drop_duplicates()

In [65]:
Genres = list(genre_df[0])

['Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Fantasy',
 'Romance',
 'Drama',
 'Action',
 'Crime',
 'Thriller',
 'Horror',
 'Mystery',
 'Sci-Fi',
 'War',
 'Musical',
 'Documentary',
 'IMAX',
 'Western',
 'Film-Noir',
 '(no genres listed)']

# User Items Recommendation


In [16]:
def do_you_mean(xxx):
    xxx = xxx.title()
    list_titles = list(movies_df.loc[movies_df.title.str.contains(xxx),'title'])
    return list_titles

In [17]:
user_item_df = pd.pivot_table(data=ratings_df, values='rating', index='userId', columns='movieId')

def similar_movies():
  xxx = input('Give us a name of a movie you like! ')

  movie_id = int(movies_df.loc[movies_df.title.str.contains(xxx),'movieId'].head(1))

  movie_ratings = user_item_df.loc[:,movie_id]
  similar_to_movie = user_item_df.corrwith(movie_ratings)

  corr_movie_id = pd.DataFrame(similar_to_movie, columns=['PearsonR'])
  corr_movie_id.dropna(inplace=True)

  rating = pd.DataFrame(ratings_df.groupby('movieId')['rating'].mean())
  rating['rating_count'] = ratings_df.groupby('movieId')['rating'].count()

  movie_corr_summary = corr_movie_id.join(rating[['rating_count','rating']])
  movie_corr_summary.drop(movie_id, inplace=True)

  topn = movie_corr_summary[movie_corr_summary['rating_count']>=movie_corr_summary['rating_count'].mean()].sort_values('PearsonR', ascending=False).head(10)

  movies_name =  movies_df[['movieId', 'title']]
  topn = topn.merge(movies_name, left_index=True, right_on="movieId")
  topn = topn.rename(columns={'title':'Title', 'rating_count':'Total Reviews', 'rating':'Rating Score'})
  topn = topn[['Title', 'Total Reviews', 'Rating Score']].set_index('Title')
  return topn.sort_values('Rating Score', ascending=False)

In [18]:
similar_movies()

/Users/alex/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:2683: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/Users/alex/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


,Total Reviews,Rating Score
Title,,
Full Metal Jacket (1987),102,4.098039
Jaws (1975),91,4.005495
"Clockwork Orange, A (1971)",120,3.995833
As Good as It Gets (1997),96,3.697917
Ed Wood (1994),70,3.678571
Speed (1994),171,3.529240
GoldenEye (1995),132,3.496212
RoboCop (1987),70,3.371429
Top Gun (1986),83,3.301205


In [19]:
def do_you_mean(xxx):
    xxx = xxx.title()
    list_titles = list(movies_df.loc[movies_df.title.str.contains(xxx),'title'])
    return list_titles

In [20]:
do_you_mean('Nemo')

['Little Nemo: Adventures in Slumberland (1992)', 'Finding Nemo (2003)']

# User Based Recommendations


In [21]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [22]:
users_items = pd.pivot_table(data=ratings_df, 
                                 values='rating', 
                                 index='userId', 
                                 columns='movieId')

users_items.fillna(0, inplace=True)

user_similarities = pd.DataFrame(cosine_similarity(users_items),
                                 columns=users_items.index, 
                                 index=users_items.index)


In [23]:
ratings_df.userId.value_counts()

414    2698
599    2478
474    2108
448    1864
274    1346
       ... 
442      20
569      20
320      20
576      20
53       20
Name: userId, Length: 610, dtype: int64

In [24]:
def recom_rest():

    user_id = int(input('Give your ID'))
    n = int(input('How many Movies should we recomend to you?'))

    weights = (
        user_similarities.query("userId!=@user_id")[user_id] / sum(user_similarities.query("userId!=@user_id")[user_id])
            )
    users_items.loc[user_id,:]==0

    not_watched_movies = users_items.loc[users_items.index!=user_id, users_items.loc[user_id,:]==0]

    weighted_averages = pd.DataFrame(not_watched_movies.T.dot(weights), columns=["predicted_rating"])

    recommendations = weighted_averages.merge(movies_df, left_index=True, right_on="movieId")
    return recommendations.sort_values("predicted_rating", ascending=False).head(n)

recom_rest()

,predicted_rating,movieId,title,genres
277,2.654727,318,"Shawshank Redemption, The (1994)",Crime|Drama
507,2.087327,589,Terminator 2: Judgment Day (1991),Action|Sci-Fi
659,1.859548,858,"Godfather, The (1972)",Crime|Drama
2078,1.663564,2762,"Sixth Sense, The (1999)",Drama|Horror|Mystery
3638,1.624820,4993,"Lord of the Rings: The Fellowship of the Ring,...",Adventure|Fantasy
123,1.585826,150,Apollo 13 (1995),Adventure|Drama|IMAX
31,1.583809,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
4800,1.502235,7153,"Lord of the Rings: The Return of the King, The...",Action|Adventure|Drama|Fantasy
4137,1.483288,5952,"Lord of the Rings: The Two Towers, The (2002)",Adventure|Fantasy
506,1.477032,588,Aladdin (1992),Adventure|Animation|Children|Comedy|Musical
